In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from plotly.graph_objs import Scatter, Layout
from plotly.offline import plot

Using TensorFlow backend.


In [2]:
def load_data(df, sequence_length=10, split=0.8):
    data_all = np.array(df).astype(float)    # 轉為浮點型別矩陣
    #print(data_all.shape) # (241,1)
    scaler = MinMaxScaler()
    data_all = scaler.fit_transform(data_all)  # 將數據縮放為 0~1 之間
    data = []
    # data 資料共有 (241-10-1)=230 筆
    for i in range(len(data_all) - sequence_length - 1):
        # 每筆 data 資料有 11 欄
        data.append(data_all[i: i + sequence_length + 1])
    reshaped_data = np.array(data).astype('float64')

    x = reshaped_data[:, :-1] # 第 1至第10個欄位為 特徵
    y = reshaped_data[:, -1]  # 第 11個欄位為 label
    #print(x.shape,y.shape) # (230,10,1) (230,1)
    split_boundary = int(reshaped_data.shape[0] * split)
    train_x = x[: split_boundary] # 前 80% 為 train 的特徵
    test_x = x[split_boundary:]   # 最後 20% 為 test 的特徵
 
    train_y = y[: split_boundary] # 前 80% 為 train 的 label
    test_y = y[split_boundary:]   # 最後 20% 為 test 的 label

    return train_x, train_y, test_x, test_y, scaler

def build_model():
    model = Sequential()     
    # 隱藏層：256 個神經元，input_shape：(10,1)
    # TIME_STEPS=10,INPUT_SIZE=1
    model.add(LSTM(input_shape=(10,1),units=256,unroll=False))
    model.add(Dense(units=1)) # 輸出層：1 個神經元
    #compile:loss, optimizer, metrics
    model.compile(loss="mse", optimizer="adam", metrics=['accuracy'])
    return model

def train_model(train_x, train_y, test_x, test_y):
    model = build_model()
    try:
        model.fit(train_x, train_y, batch_size=10000, epochs=300, validation_split=0.1)
        predict = model.predict(test_x)
        predict = np.reshape(predict, (predict.size, )) #轉換為1維矩陣
    except KeyboardInterrupt:
        print(predict)
        print(test_y)
    return predict, test_y #傳回 預測值和真實值

In [3]:
pd.options.mode.chained_assignment = None  #取消顯示pandas資料重設警告
filename = './data/Train_Y.csv'
df = pd.read_csv(filename,encoding='cp950')  #以pandas讀取檔案
ddprice=pd.DataFrame(df['Target_Y'])

In [4]:
train_x, train_y, test_x, test_y, scaler =load_data(ddprice, sequence_length=10, split=0.8)
# train_x 共 230*0.8=184 筆, test_x 共 230*0.2=46 筆
#print(train_x.shape,train_y.shape) # (184,10,1) (184,1)
#print(test_x.shape,test_y.shape)   # (46,10,1)  (46,1)
predict_y, test_y = train_model(train_x, train_y, test_x, test_y)
predict_y = scaler.inverse_transform([[i] for i in predict_y]) # 還原
test_y = scaler.inverse_transform(test_y)  # 還原

Train on 162 samples, validate on 18 samples
Epoch 1/300
162/162 [==============================] - 1s 8ms/step - loss: 0.3575 - accuracy: 0.0000e+00 - val_loss: 0.1046 - val_accuracy: 0.0000e+00
Epoch 2/300
162/162 [==============================] - 0s 994us/step - loss: 0.2519 - accuracy: 0.0000e+00 - val_loss: 0.0551 - val_accuracy: 0.0000e+00
Epoch 3/300
162/162 [==============================] - 0s 963us/step - loss: 0.1506 - accuracy: 0.0000e+00 - val_loss: 0.0210 - val_accuracy: 0.0000e+00
Epoch 4/300
162/162 [==============================] - 0s 1ms/step - loss: 0.0674 - accuracy: 0.0062 - val_loss: 0.0119 - val_accuracy: 0.0000e+00
Epoch 5/300
162/162 [==============================] - 0s 840us/step - loss: 0.0148 - accuracy: 0.0062 - val_loss: 0.0421 - val_accuracy: 0.0000e+00
Epoch 6/300
162/162 [==============================] - 0s 907us/step - loss: 0.0128 - accuracy: 0.0062 - val_loss: 0.0954 - val_accuracy: 0.0000e+00
Epoch 7/300
162/162 [==============================] 

Epoch 56/300
162/162 [==============================] - 1s 3ms/step - loss: 0.0049 - accuracy: 0.0062 - val_loss: 0.0165 - val_accuracy: 0.0000e+00
Epoch 57/300
162/162 [==============================] - 0s 1ms/step - loss: 0.0048 - accuracy: 0.0062 - val_loss: 0.0171 - val_accuracy: 0.0000e+00
Epoch 58/300
162/162 [==============================] - 0s 957us/step - loss: 0.0047 - accuracy: 0.0062 - val_loss: 0.0179 - val_accuracy: 0.0000e+00
Epoch 59/300
162/162 [==============================] - 0s 1ms/step - loss: 0.0046 - accuracy: 0.0062 - val_loss: 0.0187 - val_accuracy: 0.0000e+00
Epoch 60/300
162/162 [==============================] - 0s 1ms/step - loss: 0.0046 - accuracy: 0.0062 - val_loss: 0.0194 - val_accuracy: 0.0000e+00
Epoch 61/300
162/162 [==============================] - 1s 4ms/step - loss: 0.0046 - accuracy: 0.0062 - val_loss: 0.0199 - val_accuracy: 0.0000e+00
Epoch 62/300
162/162 [==============================] - 0s 1ms/step - loss: 0.0046 - accuracy: 0.0062 - val_lo

162/162 [==============================] - 0s 1ms/step - loss: 0.0042 - accuracy: 0.0062 - val_loss: 0.0165 - val_accuracy: 0.0000e+00
Epoch 112/300
162/162 [==============================] - 0s 2ms/step - loss: 0.0042 - accuracy: 0.0062 - val_loss: 0.0164 - val_accuracy: 0.0000e+00
Epoch 113/300
162/162 [==============================] - 0s 944us/step - loss: 0.0042 - accuracy: 0.0062 - val_loss: 0.0163 - val_accuracy: 0.0000e+00
Epoch 114/300
162/162 [==============================] - 0s 1ms/step - loss: 0.0042 - accuracy: 0.0062 - val_loss: 0.0162 - val_accuracy: 0.0000e+00
Epoch 115/300
162/162 [==============================] - 0s 1ms/step - loss: 0.0042 - accuracy: 0.0062 - val_loss: 0.0162 - val_accuracy: 0.0000e+00
Epoch 116/300
162/162 [==============================] - 0s 1ms/step - loss: 0.0042 - accuracy: 0.0062 - val_loss: 0.0161 - val_accuracy: 0.0000e+00
Epoch 117/300
162/162 [==============================] - 0s 2ms/step - loss: 0.0042 - accuracy: 0.0062 - val_loss: 0.0

Epoch 166/300
162/162 [==============================] - 0s 1ms/step - loss: 0.0038 - accuracy: 0.0062 - val_loss: 0.0150 - val_accuracy: 0.0000e+00
Epoch 167/300
162/162 [==============================] - 0s 1ms/step - loss: 0.0038 - accuracy: 0.0062 - val_loss: 0.0149 - val_accuracy: 0.0000e+00
Epoch 168/300
162/162 [==============================] - 0s 1ms/step - loss: 0.0038 - accuracy: 0.0062 - val_loss: 0.0149 - val_accuracy: 0.0000e+00
Epoch 169/300
162/162 [==============================] - 1s 7ms/step - loss: 0.0038 - accuracy: 0.0062 - val_loss: 0.0149 - val_accuracy: 0.0000e+00
Epoch 170/300
162/162 [==============================] - 0s 1ms/step - loss: 0.0038 - accuracy: 0.0062 - val_loss: 0.0149 - val_accuracy: 0.0000e+00
Epoch 171/300
162/162 [==============================] - 0s 1ms/step - loss: 0.0038 - accuracy: 0.0062 - val_loss: 0.0149 - val_accuracy: 0.0000e+00
Epoch 172/300
162/162 [==============================] - 1s 3ms/step - loss: 0.0038 - accuracy: 0.0062 - v

Epoch 221/300
162/162 [==============================] - 1s 6ms/step - loss: 0.0034 - accuracy: 0.0062 - val_loss: 0.0138 - val_accuracy: 0.0000e+00
Epoch 222/300
162/162 [==============================] - 0s 3ms/step - loss: 0.0034 - accuracy: 0.0062 - val_loss: 0.0137 - val_accuracy: 0.0000e+00
Epoch 223/300
162/162 [==============================] - 0s 877us/step - loss: 0.0034 - accuracy: 0.0062 - val_loss: 0.0137 - val_accuracy: 0.0000e+00
Epoch 224/300
162/162 [==============================] - 0s 1ms/step - loss: 0.0034 - accuracy: 0.0062 - val_loss: 0.0137 - val_accuracy: 0.0000e+00
Epoch 225/300
162/162 [==============================] - 0s 895us/step - loss: 0.0034 - accuracy: 0.0062 - val_loss: 0.0137 - val_accuracy: 0.0000e+00
Epoch 226/300
162/162 [==============================] - 0s 889us/step - loss: 0.0034 - accuracy: 0.0062 - val_loss: 0.0137 - val_accuracy: 0.0000e+00
Epoch 227/300
162/162 [==============================] - 0s 3ms/step - loss: 0.0033 - accuracy: 0.00

Epoch 276/300
162/162 [==============================] - 0s 895us/step - loss: 0.0030 - accuracy: 0.0062 - val_loss: 0.0130 - val_accuracy: 0.0000e+00
Epoch 277/300
162/162 [==============================] - 0s 914us/step - loss: 0.0030 - accuracy: 0.0062 - val_loss: 0.0129 - val_accuracy: 0.0000e+00
Epoch 278/300
162/162 [==============================] - 0s 852us/step - loss: 0.0030 - accuracy: 0.0062 - val_loss: 0.0129 - val_accuracy: 0.0000e+00
Epoch 279/300
162/162 [==============================] - 0s 2ms/step - loss: 0.0030 - accuracy: 0.0062 - val_loss: 0.0129 - val_accuracy: 0.0000e+00
Epoch 280/300
162/162 [==============================] - 0s 870us/step - loss: 0.0030 - accuracy: 0.0062 - val_loss: 0.0129 - val_accuracy: 0.0000e+00
Epoch 281/300
162/162 [==============================] - 0s 846us/step - loss: 0.0030 - accuracy: 0.0062 - val_loss: 0.0129 - val_accuracy: 0.0000e+00
Epoch 282/300
162/162 [==============================] - 0s 895us/step - loss: 0.0030 - accuracy

In [5]:
scaler

MinMaxScaler(copy=True, feature_range=(0, 1))

In [6]:
predict_y

array([[8720.39889442],
       [8736.36024165],
       [8775.3922658 ],
       [8824.95511092],
       [8879.71950154],
       [8943.44874757],
       [8993.60482745],
       [9024.58251454],
       [9021.78684179],
       [8996.58332708],
       [8985.92073233],
       [8994.84441086],
       [8996.91009807],
       [8995.34599887],
       [8997.97027627],
       [8986.10565889],
       [8959.34714356],
       [8943.46453398],
       [8956.26902622],
       [8993.04452642],
       [9047.66925005],
       [9097.04409687],
       [9129.2685218 ],
       [9157.25604442],
       [9179.61668267],
       [9179.20693581],
       [9151.37245586],
       [9136.76310219],
       [9147.02582631],
       [9166.19597595],
       [9184.23152572],
       [9186.44831146],
       [9161.59155349],
       [9120.74276977],
       [9079.79607917],
       [9037.83330345],
       [8996.14846194],
       [8952.33914243],
       [8917.74201177],
       [8913.51661876],
       [8947.47949332],
       [9000.105

In [7]:
test_y

array([[8773.55],
       [8903.68],
       [8888.07],
       [8974.76],
       [9004.86],
       [8989.18],
       [8962.6 ],
       [8891.02],
       [8912.62],
       [9049.98],
       [9034.14],
       [8918.95],
       [8980.67],
       [8982.88],
       [8884.39],
       [8859.07],
       [8963.24],
       [9078.87],
       [9091.53],
       [9122.33],
       [9116.24],
       [9122.39],
       [9165.31],
       [9187.15],
       [9117.71],
       [9034.79],
       [9175.26],
       [9225.11],
       [9206.57],
       [9187.29],
       [9128.9 ],
       [9032.16],
       [9013.07],
       [9027.33],
       [8985.63],
       [8950.91],
       [8828.36],
       [8878.63],
       [8999.52],
       [9095.  ],
       [9097.71],
       [9186.18],
       [9158.7 ],
       [9214.07],
       [9218.5 ],
       [9286.28]])

In [8]:
predict_y/test_y

array([[0.99394189],
       [0.9812078 ],
       [0.98732259],
       [0.9833082 ],
       [0.986103  ],
       [0.99491263],
       [1.00345936],
       [1.01502218],
       [1.01224857],
       [0.9940998 ],
       [0.99466255],
       [1.00850934],
       [1.00180834],
       [1.00138775],
       [1.01278425],
       [1.01433962],
       [0.99956569],
       [0.98508565],
       [0.98512231],
       [0.98582758],
       [0.99247818],
       [0.99722157],
       [0.99606762],
       [0.99674611],
       [1.00678972],
       [1.01598454],
       [0.99739653],
       [0.99042321],
       [0.99353243],
       [0.997704  ],
       [1.00606114],
       [1.01708211],
       [1.01647846],
       [1.01034777],
       [1.01047963],
       [1.00971111],
       [1.01900562],
       [1.00830186],
       [0.99091307],
       [0.98004581],
       [0.983487  ],
       [0.97974408],
       [0.98981052],
       [0.99042924],
       [0.99582816],
       [0.99276337]])

In [10]:
test_y-predict_y

array([[  53.15110558],
       [ 167.31975835],
       [ 112.6777342 ],
       [ 149.80488908],
       [ 125.14049846],
       [  45.73125243],
       [ -31.00482745],
       [-133.56251454],
       [-109.16684179],
       [  53.39667292],
       [  48.21926767],
       [ -75.89441086],
       [ -16.24009807],
       [ -12.46599887],
       [-113.58027627],
       [-127.03565889],
       [   3.89285644],
       [ 135.40546602],
       [ 135.26097378],
       [ 129.28547358],
       [  68.57074995],
       [  25.34590313],
       [  36.0414782 ],
       [  29.89395558],
       [ -61.90668267],
       [-144.41693581],
       [  23.88754414],
       [  88.34689781],
       [  59.54417369],
       [  21.09402405],
       [ -55.33152572],
       [-154.28831146],
       [-148.52155349],
       [ -93.41276977],
       [ -94.16607917],
       [ -86.92330345],
       [-167.78846194],
       [ -73.70914243],
       [  81.77798823],
       [ 181.48338124],
       [ 150.23050668],
       [ 186.074